In [ ]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

In [ ]:
%region us-west-2

In [ ]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
%idle_timeout 60

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [ ]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [ ]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [ ]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [ ]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [ ]:
from sstm_transformation.tsm_builder import TSMBuilder
from pyspark.sql.functions import input_file_name
import pyspark.sql.functions as F



In [ ]:
#Read All dfs 

df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")
df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet")
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")
df_meds = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")
df_project_vitals =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project_vitals.parquet")


In [ ]:
df_meds.count()

In [ ]:
df_project_vitals = df_project_vitals.withColumn("input_file", input_file_name())


In [ ]:
df_meds_modified = df_meds.filter(df_meds.id.isNotNull())

In [ ]:
df_meds_modified.count()

In [ ]:
df_project_vitals = df_project_vitals.withColumn('projectId', F.element_at(F.split(F.col('input_file'), '/'), -2))


In [ ]:
builder = TSMBuilder("sstm.yaml", spark=spark)


In [ ]:
df_project_vitals.show()
#s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/10387917/project_vitals.parquet

In [ ]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        
        datasink = glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame=dyf,
                        catalog_connection="dev-glue-redshift-connection",
                        connection_options={
                            "preactions" : "delete from tsm.{}".format(table_name),
                            "dbtable": "tsm.{}".format(table_name), 
                            "database": "truve",
                        },
                        redshift_tmp_dir=temp_bucket,
                        transformation_ctx="datasink",
                    )
        '''
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "XXXXXXX",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        '''
        #break
    

In [ ]:
#PeopleType Transformation
def people_type_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peopletypes(contact_df=df_contact)
    #print(result)
    upload_result(result)
    
people_type_transformation(df_contact)

In [ ]:
#Peoples Transformation
def people_master_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peoplemaster(contact_df=df_contact)
    #print(result)
    #return result
    upload_result(result)
    
people_master_result = people_master_transformation(df_contact)

In [ ]:
#Peoples Transformation
def projecttype_transformation(df_projecttype):
    #Read Raw Data
    result = builder.build_practicetypes(df_projecttype)
    print(result)
    upload_result(result)
    
projecttype_transformation(df_projecttype)

In [ ]:
#Peoples Transformation
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")

def phases_transformation(df_projecttype):
    #Read Raw Data
    
    result = builder.build_phasemaster(df_projecttype)
    print(result)
    upload_result(result)
    
phases_transformation(df_projecttype)

In [ ]:
#Peoples Transformation
def value_transformation(df_meds):
    #Read Raw Data
    result = builder.build_casefigures(df_meds)
    #print(result)
    upload_result(result)
    return result
    
result = value_transformation(df_meds_modified)

In [ ]:
#Peoples Transformation
'''
def cases_transformation(df_project):
    #Read Raw Data
    result = builder.build_casemaster(df_project)
    return result
    #print(result)
    #upload_result(result)
    
result = cases_transformation(df_project)
'''

In [ ]:
#Peoples Transformation
def intake_transformation(df_intake):
    #Read Raw Data
    result = builder.build_intakesummary(df_intake)
    return result 

result = intake_transformation(df_intake)
#result["IntakeDetails"] = result["IntakeDetails"].drop("referral_fee")
upload_result(result)


In [ ]:
#Peoples Transformation
def casesummary_transformation(df_casesummary, df_project_vitals, df_project):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary, df_project_vitals, df_project)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary, df_project_vitals, df_project)

In [ ]:
processed_project_df = result["CMS_Cases"]
processed_project_df.printSchema()

In [ ]:
#Peoples Transformation
def casetype_transformation(df_casesummary, project_df):
    #Read Raw Data
    result = builder.build_casetypes(df_casesummary, project_df)
    #print(result)
    #return result
    upload_result(result)
    return result
    
casetype_result = casetype_transformation(df_casesummary, df_project)

In [ ]:
processed_case_type_df = casetype_result["CMS_CaseTypes"]

#Peoples Transformation
def cases_transformation(df_project, processed_case_type_df, df_project_vitals, df_casesummary):
    #Read Raw Data
    result = builder.build_casemaster(df_project, processed_case_type_df, df_project_vitals, df_casesummary)
    #return result
    #print(result)
    upload_result(result)
    return result
    
result = cases_transformation(df_project, processed_case_type_df, df_project_vitals, df_casesummary)

In [ ]:
%stop_session

In [ ]:
df_casesummary.select("caseType").distinct().show(10)

In [ ]:
casetype_df = casetype_result["CMS_CaseTypes"]


In [ ]:
casetype_df.show()

In [ ]:
people_df = people_master_result["CMS_People"]

In [ ]:
people_df

In [ ]:
people_df.show()

In [ ]:

# Find count for empty, None, Null, Nan with string literals.
from pyspark.sql.functions import col,isnan,when,count
df2 = people_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in people_df.columns])
df2.show()


In [ ]:
%stop_session

In [ ]:
case_df = result["CMS_Cases"]

In [ ]:
case_figure_df.select("Case_Figure_ID", "Case_ID").show()

In [ ]:
from pyspark.sql.functions import col,isnan,when,count
df2 = case_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in case_df.columns])
df2.show()
